In [1]:
#import statements
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn imports
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import linear_model

%matplotlib inline 

#sets max columns so I can everything
pd.set_option('display.max_columns',100)
pd.set_option('display.max_colwidth', -1)

In [2]:
#load data

dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")

In [3]:
#specify if data is train/test
dfTrain['DataType'] = 'Train'
dfTest['DataType'] = 'Test'

In [4]:
#combine data frames
dfMaster = pd.concat([dfTrain,dfTest])

In [5]:
#check length of master columns
len(dfMaster.columns)

82

In [6]:
#creation of variable explanation cheat sheet
dfDesciptOnly = pd.read_csv('data_descriptions_only.csv',header =0)
dfDesciptOnly['Variable'],dfDesciptOnly['Description'] = dfDesciptOnly['Description'].str.split(':').str

In [7]:
#create df of variables only and name it "Variable"
ColumnNames = pd.DataFrame(dfMaster.columns)
ColumnNames.columns = ['Variable']

In [8]:
dfCheatSheet = pd.merge(ColumnNames,dfDesciptOnly, on = 'Variable')

In [9]:
#create filter to differentiate from Train and Test data
TrainFilter = dfMaster['DataType']=='Train'

In [10]:
#Data Cleaning

#identify & drop columns where 95% of column data is missing

#5% missing threshold
threshold5per = len(dfMaster[TrainFilter])*.05
#print(threshold95per)

#isolate columns where more than 5% of data is missing 
#can index the df.columns based on boolean value
columnsToDrop = dfMaster[TrainFilter].columns[dfMaster[TrainFilter].isnull().sum()>threshold5per]

#drop columns where more than 5% of data is missing 
dfMaster.drop(labels = columnsToDrop, inplace=True, axis =1 )

In [11]:
len(dfMaster.columns)

71

In [12]:
#Remove columns where no seperation between value
#if only have only have one or two, unlikely to provide us any extra info

In [13]:
for col in dfMaster.columns:
    if (len(dfMaster[TrainFilter][col].unique())<=4):
        print(dfMaster[TrainFilter][col].value_counts())
        print()

#for each column in df master
#if the unique values in the column is less than equal to for
#print the the value counts of the column

0.0    856
1.0    588
2.0    15 
3.0    1  
Name: BsmtFullBath, dtype: int64

0.0    1378
1.0    80  
2.0    2   
Name: BsmtHalfBath, dtype: int64

Y    1365
N    95  
Name: CentralAir, dtype: int64

Train    1460
Name: DataType, dtype: int64

TA    906
Gd    488
Ex    52 
Fa    14 
Name: ExterQual, dtype: int64

0    690
1    650
2    115
3    5  
Name: Fireplaces, dtype: int64

2    768
1    650
3    33 
0    9  
Name: FullBath, dtype: int64

0    913
1    535
2    12 
Name: HalfBath, dtype: int64

1    1392
2    65  
3    2   
0    1   
Name: KitchenAbvGr, dtype: int64

TA    735
Gd    586
Ex    100
Fa    39 
Name: KitchenQual, dtype: int64

Lvl    1311
Bnk    63  
HLS    50  
Low    36  
Name: LandContour, dtype: int64

Gtl    1382
Mod    65  
Sev    13  
Name: LandSlope, dtype: int64

Reg    925
IR1    484
IR2    41 
IR3    10 
Name: LotShape, dtype: int64

Y    1340
N    90  
P    30  
Name: PavedDrive, dtype: int64

Pave    1454
Grvl    6   
Name: Street, dtype: int64

AllPub   

In [14]:
#Based on the above analysis, drop Street and Utilities.  Neither category has enough enough for meaningful info
dfMaster.drop(labels = ['Street','Utilities'], inplace=True, axis =1 )

In [15]:
len(dfMaster.columns)

69

In [16]:
dfMaster.to_csv('dfMasterCleaned.csv')

In [17]:
#reload cleaned data

dfMasterClean = pd.read_csv('dfMasterCleaned.csv',index_col = 0)

In [34]:
dfMasterClean.dtypes.value_counts()

object     33
int64      26
float64    10
dtype: int64

In [35]:
objectFilter = list(dfMasterClean.dtypes == 'object')
intFilter = list(dfMaster.dtypes != 'object')

In [36]:
print(objectVar)

[False, False, False, False, True, True, True, False, False, True, True, False, False, True, False, True, True, True, True, True, False, True, True, True, True, False, True, False, True, False, False, False, False, True, True, True, False, False, True, True, True, False, True, True, False, False, True, False, True, False, False, True, False, False, False, True, False, True, True, True, False, True, False, False, False, False, False, False, False]


In [37]:
print(dfMasterClean.columns[objectFilter])
print(dfMasterClean.columns[intFilter])

Index(['BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'DataType',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Foundation', 'Functional', 'Heating', 'HeatingQC', 'HouseStyle',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape',
       'MSZoning', 'MasVnrType', 'Neighborhood', 'PavedDrive', 'RoofMatl',
       'RoofStyle', 'SaleCondition', 'SaleType'],
      dtype='object')
Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GrLivArea', 'HalfBath', 'Id', 'KitchenAbvGr', 'LotArea',
       'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal', 'MoSold',
       'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'SalePrice',
       'ScreenPorch', 'TotRmsAbvGrd', 'Total

In [38]:
#create list for indexing int and object variables
objectFilter = ['BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'DataType',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Foundation', 'Functional', 'Heating', 'HeatingQC', 'HouseStyle',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape',
       'MSZoning', 'MasVnrType', 'Neighborhood', 'PavedDrive', 'RoofMatl',
       'RoofStyle', 'SaleCondition', 'SaleType']

intFilter = ['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GrLivArea', 'HalfBath', 'Id', 'KitchenAbvGr', 'LotArea',
       'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal', 'MoSold',
       'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'SalePrice',
       'ScreenPorch', 'TotRmsAbvGrd', 'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt',
       'YearRemodAdd', 'YrSold']

In [39]:
dfMasterClean[intFilter]

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GrLivArea,HalfBath,Id,KitchenAbvGr,LotArea,LowQualFinSF,MSSubClass,MasVnrArea,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PoolArea,SalePrice,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,0,2,548.0,2.0,1710,1,1,1,8450,0,60,196.0,0,2,61,5,7,0,208500.0,0,8,856.0,0,2003,2003,2008
1,1262,0,0,3,978.0,0.0,0.0,1.0,284.0,0,1,2,460.0,2.0,1262,0,2,1,9600,0,20,0.0,0,5,0,8,6,0,181500.0,0,6,1262.0,298,1976,1976,2007
2,920,866,0,3,486.0,0.0,1.0,0.0,434.0,0,1,2,608.0,2.0,1786,1,3,1,11250,0,60,162.0,0,9,42,5,7,0,223500.0,0,6,920.0,0,2001,2002,2008
3,961,756,0,3,216.0,0.0,1.0,0.0,540.0,272,1,1,642.0,3.0,1717,0,4,1,9550,0,70,0.0,0,2,35,5,7,0,140000.0,0,7,756.0,0,1915,1970,2006
4,1145,1053,0,4,655.0,0.0,1.0,0.0,490.0,0,1,2,836.0,3.0,2198,1,5,1,14260,0,60,350.0,0,12,84,5,8,0,250000.0,0,9,1145.0,192,2000,2000,2008
5,796,566,320,1,732.0,0.0,1.0,0.0,64.0,0,0,1,480.0,2.0,1362,1,6,1,14115,0,50,0.0,700,10,30,5,5,0,143000.0,0,5,796.0,40,1993,1995,2009
6,1694,0,0,3,1369.0,0.0,1.0,0.0,317.0,0,1,2,636.0,2.0,1694,0,7,1,10084,0,20,186.0,0,8,57,5,8,0,307000.0,0,7,1686.0,255,2004,2005,2007
7,1107,983,0,3,859.0,32.0,1.0,0.0,216.0,228,2,2,484.0,2.0,2090,1,8,1,10382,0,60,240.0,350,11,204,6,7,0,200000.0,0,7,1107.0,235,1973,1973,2009
8,1022,752,0,2,0.0,0.0,0.0,0.0,952.0,205,2,2,468.0,2.0,1774,0,9,2,6120,0,50,0.0,0,4,0,5,7,0,129900.0,0,8,952.0,90,1931,1950,2008
9,1077,0,0,2,851.0,0.0,1.0,0.0,140.0,0,2,1,205.0,1.0,1077,0,10,2,7420,0,190,0.0,0,1,4,6,5,0,118000.0,0,5,991.0,0,1939,1950,2008


In [46]:
#identify int variables with missing values
missingIntVar = list(dfMasterClean[intFilter].columns[dfMasterClean[intFilter].isnull().sum()>0])

#remove SalePrice because need it to train on
missingIntVar = (missingIntVar[:7] +[missingIntVar[9]])

<class 'list'>


In [45]:
missingIntVar

['BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtUnfSF',
 'GarageArea',
 'GarageCars',
 'TotalBsmtSF']

In [52]:
#count the number of missing values before so see if anything is actually done

dfMasterClean[missingIntVar].isnull().sum()

BsmtFinSF1      1
BsmtFinSF2      1
BsmtFullBath    2
BsmtHalfBath    2
BsmtUnfSF       1
GarageArea      1
GarageCars      1
TotalBsmtSF     1
dtype: int64

In [48]:
#dfMasterClean[missingIntVar]

#dfMasterIntOnly[missingInt2].fillna(dfMasterIntOnly[missingInt2].mean(),inplace = True)
#dataset.fillna(dataset.mean(), inplace=True)
dfMasterClean[missingIntVar].fillna(dfMasterClean[missingIntVar].mean(),inplace = True)



//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,GarageArea,GarageCars,TotalBsmtSF
0,706.0,0.0,1.0,0.0,150.0,548.0,2.0,856.0
1,978.0,0.0,0.0,1.0,284.0,460.0,2.0,1262.0
2,486.0,0.0,1.0,0.0,434.0,608.0,2.0,920.0
3,216.0,0.0,1.0,0.0,540.0,642.0,3.0,756.0
4,655.0,0.0,1.0,0.0,490.0,836.0,3.0,1145.0
5,732.0,0.0,1.0,0.0,64.0,480.0,2.0,796.0
6,1369.0,0.0,1.0,0.0,317.0,636.0,2.0,1686.0
7,859.0,32.0,1.0,0.0,216.0,484.0,2.0,1107.0
8,0.0,0.0,0.0,0.0,952.0,468.0,2.0,952.0
9,851.0,0.0,1.0,0.0,140.0,205.0,1.0,991.0


In [49]:
dfMasterClean[missingIntVar].isnull().sum()

BsmtFinSF1      1
BsmtFinSF2      1
BsmtFullBath    2
BsmtHalfBath    2
BsmtUnfSF       1
GarageArea      1
GarageCars      1
TotalBsmtSF     1
dtype: int64

In [110]:
dfMasterIntOnly[missingInt2].fillna(dfMasterIntOnly[missingInt2].mean(),inplace = True)
#dataset.fillna(dataset.mean(), inplace=True)

C:\Users\jbonanno\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,GarageArea,GarageCars,TotalBsmtSF
0,706.0,0.0,1.0,0.0,150.0,548.0,2.0,856.0
1,978.0,0.0,0.0,1.0,284.0,460.0,2.0,1262.0
2,486.0,0.0,1.0,0.0,434.0,608.0,2.0,920.0
3,216.0,0.0,1.0,0.0,540.0,642.0,3.0,756.0
4,655.0,0.0,1.0,0.0,490.0,836.0,3.0,1145.0
5,732.0,0.0,1.0,0.0,64.0,480.0,2.0,796.0
6,1369.0,0.0,1.0,0.0,317.0,636.0,2.0,1686.0
7,859.0,32.0,1.0,0.0,216.0,484.0,2.0,1107.0
8,0.0,0.0,0.0,0.0,952.0,468.0,2.0,952.0
9,851.0,0.0,1.0,0.0,140.0,205.0,1.0,991.0


In [111]:
dfMasterIntOnly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 36 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
BedroomAbvGr     2919 non-null int64
BsmtFinSF1       2918 non-null float64
BsmtFinSF2       2918 non-null float64
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtUnfSF        2918 non-null float64
EnclosedPorch    2919 non-null int64
Fireplaces       2919 non-null int64
FullBath         2919 non-null int64
GarageArea       2918 non-null float64
GarageCars       2918 non-null float64
GrLivArea        2919 non-null int64
HalfBath         2919 non-null int64
Id               2919 non-null int64
KitchenAbvGr     2919 non-null int64
LotArea          2919 non-null int64
LowQualFinSF     2919 non-null int64
MSSubClass       2919 non-null int64
MasVnrArea       2896 non-null float64
MiscVal          2919 non-null int64
MoSold           2919 non-

In [76]:
dfMasterIntOnly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 36 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
BedroomAbvGr     2919 non-null int64
BsmtFinSF1       2918 non-null float64
BsmtFinSF2       2918 non-null float64
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtUnfSF        2918 non-null float64
EnclosedPorch    2919 non-null int64
Fireplaces       2919 non-null int64
FullBath         2919 non-null int64
GarageArea       2918 non-null float64
GarageCars       2918 non-null float64
GrLivArea        2919 non-null int64
HalfBath         2919 non-null int64
Id               2919 non-null int64
KitchenAbvGr     2919 non-null int64
LotArea          2919 non-null int64
LowQualFinSF     2919 non-null int64
MSSubClass       2919 non-null int64
MasVnrArea       2896 non-null float64
MiscVal          2919 non-null int64
MoSold           2919 non-